In [1]:
from datasets import load_dataset

dataset_pal = load_dataset("imagefolder", data_dir="../data/filtered/pal", drop_labels=True, split="train")
print(dataset_pal)
dataset_pokemon = load_dataset("imagefolder", data_dir="../data/filtered/pokemon", drop_labels=True, split="train")
print(dataset_pokemon)

c:\Users\hiroga\miniconda3\envs\pokemon-palworld-v2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['image'],
    num_rows: 34945
})
Dataset({
    features: ['image'],
    num_rows: 48837
})


In [2]:
dataset_pal = dataset_pal.map(lambda example: {'label': 0})
dataset_pokemon = dataset_pokemon.map(lambda example: {'label': 1})

In [3]:
from datasets import concatenate_datasets

dataset = concatenate_datasets([dataset_pal, dataset_pokemon])

In [4]:
from torch.utils.data import DataLoader
from torchvision import transforms

compose = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.convert("RGB")),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transformed = dataset.map(lambda example: {"image": compose(example["image"])}, batched=False)
transformed.set_format("torch")
train_test_dataset = transformed.train_test_split(test_size=0.2)

In [5]:
batch_size = 128
train_dataloader = DataLoader(train_test_dataset["train"], batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(train_test_dataset["test"], batch_size=batch_size, shuffle=False)

In [6]:
label = ["pal", "pokemon"]

In [7]:
import torch
import wandb

def train(model, optimizer, criterion, train_loader, test_loader, num_epochs, device):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_correct = 0
        total = 0
        
        for batch in train_loader:
            images, labels = batch["image"], batch["label"]
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_correct += torch.sum(predicted == labels.data)
            total += len(labels)
        
        model.eval()
        running_test_loss = 0.0
        running_test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for batch in train_loader:
                images, labels = batch["image"], batch["label"]
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                test_total += len(labels)

                running_test_loss += loss.item()
                running_test_correct += torch.sum(predicted == labels.data)
        
        log = {
            "epoch": epoch +1,
            "train_loss": running_loss / len(train_loader),
            "train_acc": running_correct / total,
            "test_loss": running_test_loss / len(test_loader),
            "test_acc": running_test_correct / test_total
        }
        print(log)
        wandb.log(log)


In [8]:
# Fine Tuning from ResNet18
import torchvision.models as models
import wandb
from datetime import datetime
from safetensors.torch import save_file

model_name = "ResNet18_FineTuned"
last_layer_learning_rate = 0.01
last_layer_momentum = 0.9
last_layer_epoches = 5
full_layer_learning_rate = 0.001
full_layer_momentum = 0.001
full_layer_epoches = 5

wandb.init(
    project="pokemon-palworld",
    config={
        "model_name": model_name,
        "labels": label,
        "last_layer_learning_rate": last_layer_learning_rate,
        "last_layer_momentum": last_layer_momentum,
        "last_layer_epochs": last_layer_epoches,
        "full_layer_learning_rate": full_layer_learning_rate,
        "full_layer_momentum": full_layer_momentum,
        "full_layer_epochs": full_layer_epoches,
        "architecture": "CNN",
        "dataset": "pokemon-palworld",
        "train_size": len(train_dataloader.dataset),
        "test_size": len(test_dataloader.dataset),
        "batch_size": batch_size,
    }
)

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = torch.nn.Linear(model.fc.in_features, len(label))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

criterion = torch.nn.CrossEntropyLoss()

# Fine-tune the last layer for a few epochs
optimizer = torch.optim.SGD(model.fc.parameters(), lr=last_layer_learning_rate, momentum=last_layer_momentum)
train(model, optimizer, criterion, train_dataloader, test_dataloader, num_epochs=last_layer_epoches, device=device)

# Unfreeze all the layers and fine-tune the entire network for a few more epochs
for param in model.parameters():
    param.requires_grad = True
optimizer = torch.optim.SGD(model.parameters(), lr=full_layer_learning_rate, momentum=full_layer_momentum)
train(model, optimizer, criterion, train_dataloader, test_dataloader, num_epochs=full_layer_epoches, device=device)

save_file(model.state_dict(), f"../models/snapshots/{model_name}_epoch{last_layer_epoches}_{full_layer_epoches}_{datetime.now().strftime('%Y%m%d%H%M%S')}.safetensors")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hiroga. Use `wandb login --relogin` to force relogin


c:\Users\hiroga\miniconda3\envs\pokemon-palworld-v2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hiroga\miniconda3\envs\pokemon-palworld-v2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


{'epoch': 1, 'train_loss': 0.05833915864536894, 'train_acc': tensor(0.9789, device='cuda:0'), 'test_loss': 0.11876845688253414, 'test_acc': tensor(0.9912, device='cuda:0')}
{'epoch': 2, 'train_loss': 0.03478731791966929, 'train_acc': tensor(0.9890, device='cuda:0'), 'test_loss': 0.10162574984133244, 'test_acc': tensor(0.9923, device='cuda:0')}
{'epoch': 3, 'train_loss': 0.03210982568926384, 'train_acc': tensor(0.9901, device='cuda:0'), 'test_loss': 0.09957705784894753, 'test_acc': tensor(0.9918, device='cuda:0')}
{'epoch': 4, 'train_loss': 0.029581266190529667, 'train_acc': tensor(0.9904, device='cuda:0'), 'test_loss': 0.11058470348379652, 'test_acc': tensor(0.9923, device='cuda:0')}
{'epoch': 5, 'train_loss': 0.029816618186993993, 'train_acc': tensor(0.9903, device='cuda:0'), 'test_loss': 0.08752415181166058, 'test_acc': tensor(0.9933, device='cuda:0')}
{'epoch': 1, 'train_loss': 0.020498616995582294, 'train_acc': tensor(0.9938, device='cuda:0'), 'test_loss': 0.05274752890984421, 'tes

In [10]:
# SimpleCNN
import sys

sys.path.append('..')

import torch
import wandb
from safetensors.torch import save_file

from src.SimpleCNN import SimpleCNN

model_name = "SimpleCNN"
learning_rate = 0.001
epochs = 5
image_size = 256

wandb.init(
    project="pokemon-palworld",
    config={
        "model_name": model_name,
        "learning_rate": learning_rate,
        "architecture": "CNN",
        "dataset": "pokemon-palworld",
        "epochs": epochs,
        "image_size": image_size,
        "train_size": len(train_dataloader.dataset),
        "test_size": len(test_dataloader.dataset),
        "batch_size": batch_size,
    }
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(image_size=image_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

train(model, optimizer, criterion, train_dataloader, test_dataloader, num_epochs=epochs, device=device)

save_file(model.state_dict(), f"../models/snapshots/{model_name}_epoch{epochs}.safetensors")

epoch,▁▃▅▆█▁▃▅▆█
test_acc,▁▂▂▂▃▆▇▇██
test_loss,█▇▇▇▆▃▂▂▁▁
train_acc,▁▅▅▅▅▇▇███
train_loss,█▅▄▄▄▃▂▁▁▁
epoch,5
test_acc,0.9989
test_loss,0.02106
train_acc,0.99773
train_loss,0.00814


ValueError: Expected input batch_size (98) to match target batch_size (128).